In [ ]:
###################################################################################################
#
# appendix_bs.py 	    (c) Ian Williams, Adnan Khan, Matt McQuinn
#     				    	        ianw89@live.com
#
#
#  This file generates the plot in Appendix B of Khan, Williams, McQuinn 2022
#
###################################################################################################

from cgmbrush.cgmbrush import *
from cgmbrush.plots.plots import *
import matplotlib.pyplot as plt

curve_thickness = 2

plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=XBIG_SIZE)     # fontsize of the axes title
plt.rc('legend', fontsize=BIGGER_SIZE)    # legend fontsize
plt.rcParams['legend.title_fontsize'] = BIGGER_SIZE

seed = '50g89Gfh03f4Gh0r38h2TfM08' # a consistent random seed to use 

# TODO refactor this to share code with redshift_stacking_plots.ipynb

In [ ]:
# Load or Generate the Convolution Results needed
provider = BolshoiProvider()
resolution = 8
grid_size = resolution * 1024
load_data = True
data_date = '2022-05-12'
redshifts = RS_array_gen(1, provider.Lbox)

configs = [
    Configuration(MassDependentProfile(SphericalTophatProfile(rvir_factor=2), NFWProfile(), 10**10.5), provider=provider, resolution=resolution),
    Configuration(MassDependentProfile(SphericalTophatProfile(rvir_factor=2), NFWProfile(), 10**11.5), provider=provider, resolution=resolution),
    Configuration(MassDependentProfile(SphericalTophatProfile(rvir_factor=2), NFWProfile(), 10**12.5), provider=provider, resolution=resolution),
    Configuration(MassDependentProfile(SphericalTophatProfile(rvir_factor=2), NFWProfile(), 10**13.5), provider=provider, resolution=resolution),
]

for c in configs:
    c.RS_array = redshifts
    c.seed = seed
    c.datestamp = data_date 

    c.run(load_from_files=load_data)
    c.generate_stacked_fields(load_from_files=load_data)
    
    # These summed results is all we need for plots
    c.plot_name = "{:.1f} $M_\odot$".format(math.log10(c.addition_profile.cutoff_mass))
    c.sum_to_01 = sum(c.get_stacked_final_field()[0:1,:,:])
    c.sum_to_05 = sum(c.get_stacked_final_field()[0:5,:,:])
    c.sum_to_1 = sum(c.get_stacked_final_field())
    c.hist_tenth= create_histograms(c.get_stacked_final_field()[0:1,:,:], grid_size)
    c.hist_half = create_histograms(c.get_stacked_final_field()[0:5,:,:], grid_size)
    c.hist_one = create_histograms(c.get_stacked_final_field(), grid_size)

    c.clear_results()

    

In [ ]:
# Makes the figure of the 3 histograms out to 3 redshifts for the varios tophat to NFW change-overs
fig, axs = plt.subplots(1, 3, sharex='col', sharey='row',
                        gridspec_kw={'hspace': 0, 'wspace': 0},figsize=(20,20*15 / 45))

curve_lw = curve_thickness

for c in configs:
    axs[0].plot(c.hist_tenth[0] ,c.hist_tenth[0]*c.hist_tenth[1], '-', label=c.plot_name, lw=curve_lw)
    axs[1].plot(c.hist_half[0] ,c.hist_half[0]*c.hist_half[1], '-', label=c.plot_name, lw=curve_lw)
    axs[2].plot(c.hist_one[0] ,c.hist_one[0]*c.hist_one[1], '-', label=c.plot_name, lw=curve_lw)

# Axis range
axs[0].set_xlim(xmin=0,xmax=199)
axs[1].set_xlim(xmin=0,xmax=1499)
axs[2].set_xlim(xmin=0,xmax=2500)

# Subplot titles
axs[0].set_title('Redshift = 0.1') 
axs[1].set_title('Redshift = 0.5')
axs[2].set_title('Redshift = 1.0')

# Mean DM: set for the given redshift
mean=axs[0].axvline(np.mean(configs[0].sum_to_01), color='k', linestyle='-', linewidth=0.5)
axs[1].axvline(np.mean(configs[0].sum_to_05), color='k', linestyle='-', linewidth=0.5)
axs[2].axvline(np.mean(configs[0].sum_to_1), color='k', linestyle='-', linewidth=0.5)
#mean.set_label('Mean DM')

# Axis labels
axs[1].set_xlabel('DM [pc $cm^{-3}$]', fontsize=XBIG_SIZE)
axs[0].set_ylabel(r'$ P(DM) \times DM$', fontsize=XBIG_SIZE)

# Size of ticks
axs[0].tick_params(axis='both', which='major', length=5, width=1)
axs[1].tick_params(axis='both', which='major', length=5, width=1)
axs[2].tick_params(axis='both', which='major', length=5, width=1)

# Hide y ticks in 2nd and 3rd panes
axs[1].axes.yaxis.set_visible(False)
axs[2].axes.yaxis.set_visible(False)

# legend
axs[0].legend(loc='upper right', frameon=False, title='STH to NFW at')


saveFig('STH_NFW_transition.pdf', fig, bbox_inches='tight')